### Inter-rater agreement kalkulieren

Dieses Notebook kalkuliert das inter-rater agreement aus einer Annotationsstudie, die von den Teilnehmer:innen der Lehrveranstaltung im WS 2023 durchgeführt wurde. Die Excel-Datei "newspaper_sentiment_migration.csv" enthält 24 Sätze aus den Zeitungen "Das Vaterland" (Zeitung für die österreichische Monarchie, 1860-1911) und "Neue Freie Presse" (1864-1939). Es wurde jeweils der Satz aus der Spalte "To Annotate" mit einem Polaritätswert (pos, neu oder neg) annotiert. Um den Satz im Kontext lesen zu können, wird auch der vorherige bzw. nachfolgende Satz zur Verfügung gestellt, sofern dieser existiert. 

Projektkontext: Lucija Krusic untersucht in ihrer Dissertation den Sentiment gegenüber MigrantInnen in österreichischen Zeitungen. 

- **Inter-rater agreement**: Berechnet die Übereinstimmung der Annotationen zwischen verschiedenen Annotator:innen. 
- **Intra-rater agreement**: Bezieht sich auf die Konsistenz mehrerer Annotationsdurchläufe einer einzigen Person über die Zeit.

Metriken zur Kalkulation von Übereinstimmungen: 
- **Cohen's kappa**: Cohens Kappa ist eine statistische Maßzahl, die verwendet wird, um das Ausmaß der Übereinstimmung (Agreement) zwischen zwei Bewertungen zu messen. Cohens Kappa wird häufig bei kategorialen Daten verwendet. 

Weitere Metriken sind: **Fleiss' kappa** oder **Krippendorf's Alpha**

Cohen's kappa agreements:
 - < 0.00 = Poor
 - 0.00-0.20 = Slight
 - 0.20-0.40 = Fair
 - 0.41-0.60 = Moderate
 - 0.61-0.80 = Substantial
 - 0.81-1.00 = Almost perfect



## Dateien importieren

1. Import des glob-Moduls für die Suche nach Dateien mit einem bestimmten Namensmuster innerhalb eines Verzeichnisses.
2. Pfad zum Folder definieren in der Variable `folder_path`
3. Die Funktion `glob.glob()`liefert eine Liste von Dateipfaden zurück, die einem bestimmten Muster entsprechen. 

In [ ]:
import glob
import pandas as pd

folder_path = "annotationen/"

file_paths = glob.glob(folder_path + "*.xlsx")

Im nächsten Schritt werden 
- Dateipfade identifiziert um zu überprüfen, ob die Daten überhaupt erreichbar sind
- Ein leeres DataFrame erstellt um Daten aus allen Dateien kombinieren zu können
- Jedes Excelfile eingelesen und der inhalt dem `combined_data` Dateframe hinzugefügt. 

In [ ]:
print("Identified file paths:", file_paths)

combined_data = pd.DataFrame()

for file in file_paths:
    df = pd.read_excel(file)


In [ ]:
df

Exkurs: Lädt die Daten und überprüft, ob der Wert in der Spalte Polarität valide ist.

In [ ]:
data_frames = []
invalid_values = {}

for file_path in file_paths:
    df = pd.read_excel(file_path)
    data_frames.append(df)

    invalid_entries = df[~df["Polarität (pos/neu/neg)"].isin(["neg", "pos", "neu"])]
    if not invalid_entries.empty:
        invalid_values[os.path.basename(file_path)] = invalid_entries

invalid_values

In [ ]:
from sklearn.metrics import cohen_kappa_score
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Extrahiert die Sätze und die korrespondierenden Annotationen

In [ ]:
sentences_annotations = {}
for df in data_frames:
    for index, row in df.iterrows():
        sentence = row["To annotate"]
        annotation = row["Polarität (pos/neu/neg)"]
        if sentence not in sentences_annotations:
            sentences_annotations[sentence] = []
        sentences_annotations[sentence].append(annotation)

# Filtert Sätze, die nicht von allen annotiert wurden. 
fully_annotated_sentences = {sentence: annotations for sentence, annotations in sentences_annotations.items() if len(annotations) == len(data_frames)}



In [ ]:
# Kalkuliert Cohen's kappa für jedes Paar an Annotator:innen
annotators = range(len(data_frames))
kappa_scores = np.zeros((len(data_frames), len(data_frames)))

for i, j in itertools.combinations(annotators, 2):
    annotations_i = [fully_annotated_sentences[sentence][i] for sentence in fully_annotated_sentences]
    annotations_j = [fully_annotated_sentences[sentence][j] for sentence in fully_annotated_sentences]
    kappa = cohen_kappa_score(annotations_i, annotations_j)
    kappa_scores[i][j] = kappa
    kappa_scores[j][i] = kappa


In [ ]:
# Visualisiert die Resultate
annotator_labels = [f'Annotator {i+1}' for i in annotators]
sns.heatmap(kappa_scores, annot=True, fmt=".2f", xticklabels=annotator_labels, yticklabels=annotator_labels)
plt.title("Cohen's Kappa Scores Between Annotators")
plt.show()

# Berechnet durchschnittlichen Cohen's Kappa-Wert
average_kappa = np.sum(kappa_scores) / (len(annotators) * (len(annotators) - 1))
average_kappa
